<a href="https://colab.research.google.com/github/LikeWind99/DeepLearning/blob/master/%E4%BA%8C%E5%88%86%E7%B1%BB1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import torch as t
import torchvision
from torchvision.datasets import ImageFolder
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torch.nn.functional as F
import tqdm
import numpy as np

In [17]:
normalize = T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
transform = T.Compose([
    T.CenterCrop(224),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    normalize
])
device = t.device('cuda' if t.cuda.is_available() else 'cpu')
batch_size = 4
num_works = 2
epochs = 100


In [18]:
dataset = ImageFolder(r'/content/drive/MyDrive/test', transform=transform)
sample_size = len(dataset)
trainDataset, testDataset = t.utils.data.random_split(dataset= dataset, lengths=[int(0.8 * sample_size), int(0.2 * sample_size)])

trainLoader = DataLoader(dataset=trainDataset, batch_size=batch_size, shuffle=True, drop_last=True)
testLoader = DataLoader(dataset=testDataset, batch_size=batch_size, drop_last=True)

In [19]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1, shortcut=None):
        super(ResidualBlock,self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel,outchannel,3,stride, 1,bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel,outchannel,3,1,1,bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.right = shortcut
    
    def forward(self,x):
        out = self.left(x)
        residual = x if self.right is None else self.right(x)
        out += residual
        return F.relu(out)
    
class ResNet(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNet,self).__init__()
        self.pre = nn.Sequential(
                nn.Conv2d(3, 64, 7, 2, 3, bias=False),
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(3, 2, 1)
        )
        self.layer1 = self._make_layer( 64, 64, 3)
        self.layer2 = self._make_layer( 64, 128, 4, stride=2)
        self.layer3 = self._make_layer( 128, 256, 6, stride=2)
        self.layer4 = self._make_layer( 256, 512, 3, stride=2)
        self.fc = nn.Linear(512, num_classes)
        
    def _make_layer(self,  inchannel, outchannel, block_num, stride=1):
        shortcut = nn.Sequential(
                nn.Conv2d(inchannel,outchannel,1,stride, bias=False),
                nn.BatchNorm2d(outchannel)
        )
        layers = []
        layers.append(ResidualBlock(inchannel, outchannel, stride, shortcut))
        
        for i in range(1, block_num):
            layers.append(ResidualBlock(outchannel, outchannel))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.pre(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = F.avg_pool2d(x, 7)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [20]:
net = ResNet()
total_step = len(trainLoader)
criterion = nn.CrossEntropyLoss()
optimizer = t.optim.Adam(params=net.parameters())
for epoch in range(epochs):
  for i, (imgs, labels) in tqdm.tqdm(enumerate(trainLoader)):
    net = net.to(device)
    optimizer.zero_grad()
    imgs = imgs.to(device)
    labels = labels.to(device)
    predict = net(imgs)
    loss = criterion(predict, labels)
    loss.backward()
    optimizer.step()
    if (i + 1) % 10 == 0:
      print(f'Epoch [{epoch}/{epochs}], Step [{i+1}/{total_step}], loss: {loss.item():.4f}')





0it [00:00, ?it/s]


2it [00:00, 13.00it/s]


4it [00:00, 14.38it/s]


7it [00:00, 16.39it/s]


10it [00:00, 17.77it/s]


13it [00:00, 19.30it/s]

Epoch [0/100], Step [10/187], loss: 0.0099





16it [00:00, 20.48it/s]


19it [00:00, 21.26it/s]


22it [00:01, 21.33it/s]


25it [00:01, 22.20it/s]

Epoch [0/100], Step [20/187], loss: 0.1016





28it [00:01, 22.92it/s]


31it [00:01, 22.76it/s]


34it [00:01, 23.23it/s]

...




165it [00:07, 23.00it/s]


168it [00:07, 23.36it/s]


171it [00:07, 23.08it/s]


174it [00:07, 23.23it/s]

Epoch [99/100], Step [170/187], loss: 0.0000





177it [00:07, 23.53it/s]


180it [00:07, 23.13it/s]


183it [00:07, 23.48it/s]

Epoch [99/100], Step [180/187], loss: 0.0108





187it [00:08, 23.23it/s]


In [25]:
with t.no_grad():
    correct = 0
    total = 0
    for img, label in testLoader:
        img = img.to(device)
        label = label.to(device)
        outputs = net(img)
        _, predicted = t.max(outputs.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
        print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 100.0 %
Accuracy of the model on the test images: 100.0 %
Accuracy of the model on the test images: 100.0 %
Accuracy of the model on the test images: 100.0 %
Accuracy of the model on the test images: 100.0 %
Accuracy of the model on the test images: 100.0 %
Accuracy of the model on the test images: 96.42857142857143 %
Accuracy of the model on the test images: 96.875 %
Accuracy of the model on the test images: 94.44444444444444 %
Accuracy of the model on the test images: 95.0 %
Accuracy of the model on the test images: 95.45454545454545 %
Accuracy of the model on the test images: 95.83333333333333 %
Accuracy of the model on the test images: 96.15384615384616 %
Accuracy of the model on the test images: 96.42857142857143 %
Accuracy of the model on the test images: 96.66666666666667 %
Accuracy of the model on the test images: 96.875 %
Accuracy of the model on the test images: 97.05882352941177 %
Accuracy of the model on the test images: 97.22222222